# Teil 9 - Einführung in verschlüsselte Programme

Ob Sie es glauben oder nicht, es ist möglich, mit verschlüsselten Daten zu rechnen. Mit anderen Worten, es ist möglich, ein Programm auszuführen, in dem ** ALLE Variablen ** im Programm ** verschlüsselt ** sind!

In diesem Tutorial werden wir uns mit den grundlegenden Werkzeugen der verschlüsselten Berechnung befassen. Insbesondere konzentrieren wir uns auf einen beliebten Ansatz namens Secure Multi-Party Computation. In dieser Lektion lernen wir, wie Sie einen verschlüsselten Taschenrechner erstellen, der Berechnungen für verschlüsselte Zahlen durchführen kann.

Autoren:
- Andrew Trask - Twitter: [@iamtrask](https://twitter.com/iamtrask)
- Théo Ryffel - GitHub: [@LaRiffle](https://github.com/LaRiffle)

Verweise:
- Morten Dahl - [Blog](https://mortendahl.github.io) - Twitter: [@mortendahlcs](https://twitter.com/mortendahlcs)

Übersetzer:
- Vineet Jain - Github: [@vineetjai](https://github.com/vineetjai)

# Schritt 1: Verschlüsselung mit sicherer Mehrparteienberechnung

SMPC ist auf den ersten Blick eine ziemlich seltsame Form der "Verschlüsselung". Anstatt einen öffentlichen / privaten Schlüssel zum Verschlüsseln einer Variablen zu verwenden, wird jeder Wert in mehrere "Freigaben" aufgeteilt, von denen jede wie ein privater Schlüssel funktioniert. In der Regel werden diese "Aktien" auf zwei oder mehr Eigentümer verteilt. Um die Variable zu entschlüsseln, müssen alle Eigentümer zustimmen, die Entschlüsselung zuzulassen. Im Wesentlichen hat jeder einen privaten Schlüssel.

### Verschlüsseln()

Nehmen wir also an, wir wollten eine Variable `x` "verschlüsseln". Wir könnten dies auf folgende Weise tun.

 > Die Verschlüsselung verwendet keine Gleitkommazahlen oder reellen Zahlen, sondern erfolgt in einem mathematischen Raum namens [Integer Quotient Ring](http://mathworld.wolfram.com/QuotientRing.html), bei dem es sich im Grunde um die Ganzzahlen zwischen `0` und` Q- 1`, wobei `Q` eine Primzahl und" groß genug "ist, damit der Raum alle Zahlen enthalten kann, die wir in unseren Experimenten verwenden. In der Praxis machen wir bei einem Wert `x` eine Ganzzahl `x% Q`, um in den Ring zu passen. (Deshalb vermeiden wir die Verwendung der Zahl `x`> `Q`).

In [1]:
Q = 1234567891011

In [2]:
x = 25

In [3]:
import random

def encrypt(x):
    share_a = random.randint(-Q,Q)
    share_b = random.randint(-Q,Q)
    share_c = (x - share_a - share_b) % Q
    return (share_a, share_b,  share_c)

In [4]:
encrypt(x)

(-1164697238631, -8486745031, 1173183983687)

Wie Sie hier sehen können, haben wir unsere Variable "x" in 3 verschiedene Aktien aufgeteilt, die an 3 verschiedene Eigentümer gesendet werden können.

### Decrypt ()

Wenn wir diese 3 Freigaben entschlüsseln wollten, könnten wir sie einfach zusammenfassen und den Modul des Ergebnisses (mod Q) nehmen.

In [5]:
def decrypt(*shares):
    return sum(shares) % Q

In [6]:
a,b,c = encrypt(25)

In [7]:
decrypt(a,b,c)

25

Beachten Sie, dass die Entschlüsselung nicht funktioniert, wenn wir versuchen, mit nur zwei Freigaben zu entschlüsseln!

In [8]:
decrypt(a, b)

566337794850

Daher müssen alle Eigentümer teilnehmen, um den Wert zu entschlüsseln. Auf diese Weise verhalten sich die "Freigaben" wie private Schlüssel, die alle vorhanden sein müssen, um einen Wert zu entschlüsseln.

# Schritt 2: Grundlegende Arithmetik mit SMPC

Die wirklich außergewöhnliche Eigenschaft von Secure Multi-Party Computation ist jedoch die Fähigkeit, Berechnungen durchzuführen **während die Variablen noch verschlüsselt sind**. Lassen Sie uns unten eine einfache Ergänzung demonstrieren.

In [9]:
x = encrypt(25)
y = encrypt(5)

In [10]:
def add(x, y):
    z = list()
    # the first worker adds their shares together
    z.append((x[0] + y[0]) % Q)
    
    # the second worker adds their shares together
    z.append((x[1] + y[1]) % Q)
    
    # the third worker adds their shares together
    z.append((x[2] + y[2]) % Q)
    
    return z

In [11]:
decrypt(*add(x,y))

30

### Erfolg !!!

Und da hast du es! Wenn jeder Mitarbeiter (separat) seine Anteile addiert, werden die resultierenden Anteile auf den richtigen Wert (25 + 5 == 30) entschlüsselt.

Wie sich herausstellt, gibt es SMPC-Protokolle, die diese verschlüsselte Berechnung für die folgenden Operationen ermöglichen können:
- zusätzlich (was wir gerade gesehen haben)
- Multiplikation
- Vergleich

und mit diesen grundlegenden Grundelementen können wir beliebige Berechnungen durchführen !!!

Im nächsten Abschnitt erfahren Sie, wie Sie die PySyft-Bibliothek verwenden, um diese Vorgänge auszuführen!

# Schritt 3: SMPC mit PySyft

In den vorherigen Abschnitten haben wir einige grundlegende Intuitionen skizziert, die SMPC funktionieren soll. In der Praxis möchten wir jedoch nicht alle primitiven Operationen selbst von Hand schreiben müssen, wenn wir unsere verschlüsselten Programme schreiben. In diesem Abschnitt werden wir uns mit den Grundlagen der verschlüsselten Berechnung mit PySyft befassen. Insbesondere werden wir uns darauf konzentrieren, wie die drei zuvor genannten Grundelemente ausgeführt werden: Addition, Multiplikation und Vergleich.

Zuerst müssen wir einige virtuelle Arbeiter erstellen (mit denen Sie hoffentlich jetzt in unseren vorherigen Tutorials vertraut sind).

In [12]:
import torch
import syft as sy
hook = sy.TorchHook(torch)

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
bill = sy.VirtualWorker(hook, id="bill")

### Grundlegende Verschlüsselung / Entschlüsselung

Die Verschlüsselung ist so einfach wie das Aufnehmen eines PySyft-Tensors und das Aufrufen von .share(). Die Entschlüsselung ist so einfach wie das Aufrufen von .get() für die gemeinsam genutzte Variable

In [13]:
x = torch.tensor([25])

In [14]:
x

tensor([25])

In [15]:
encrypted_x = x.share(bob, alice, bill)

In [16]:
encrypted_x.get()

tensor([25])

In [17]:
bob._objects

{}

In [18]:
x = torch.tensor([25]).share(bob, alice, bill)

In [19]:
# Bob's share
bobs_share = list(bob._objects.values())[0]
bobs_share

tensor([533229672735976194])

In [20]:
# Alice's share
alices_share = list(alice._objects.values())[0]
alices_share

tensor([1459167363511690327])

In [21]:
# Bill's share
bills_share = list(bill._objects.values())[0]
bills_share

tensor([-1992397036247666496])

Und wenn wir wollten, könnten wir diese Werte mit dem gleichen Ansatz entschlüsseln, über den wir zuvor gesprochen haben !!!

In [22]:
Q = x.child.field

(bobs_share + alices_share + bills_share) % Q

tensor([25])

Wie Sie sehen können, hat es beim Aufrufen von `.share()` den Wert einfach in 3 Aktien aufgeteilt und eine Aktie an jede der Parteien gesendet!

# Verschlüsselte Arithmetik

Und jetzt sehen Sie, dass wir die zugrunde liegenden Werte rechnen können! Die API ist so aufgebaut, dass wir einfach wie normale PyTorch-Tensoren arithmetisch arbeiten können.

In [23]:
x = torch.tensor([25]).share(bob,alice)
y = torch.tensor([5]).share(bob,alice)

In [24]:
z = x + y
z.get()

tensor([30])

In [25]:
z = x - y
z.get()

tensor([20])

# Verschlüsselte Multiplikation

Für die Multiplikation benötigen wir eine zusätzliche Partei, die für die konsistente Erzeugung von Zufallszahlen verantwortlich ist (und nicht mit einer der anderen Parteien zusammenarbeitet). Wir nennen diese Person einen "crypto provider". Für alle intensiven Zwecke ist der Krypto-Anbieter nur ein zusätzlicher VirtualWorker. Es ist jedoch wichtig zu erkennen, dass der Krypto-Anbieter kein "owner" ist, da er keine Freigaben besitzt, sondern jemand, dem man vertrauen muss, um keine Absprachen zu treffen mit einem der bestehenden Aktionäre.

In [26]:
crypto_provider = sy.VirtualWorker(hook, id="crypto_provider")

In [27]:
x = torch.tensor([25]).share(bob,alice, crypto_provider=crypto_provider)
y = torch.tensor([5]).share(bob,alice, crypto_provider=crypto_provider)

In [28]:
# multiplication

z = x * y
z.get()

tensor([125])

Sie können auch eine Matrixmultiplikation durchführen

In [29]:
x = torch.tensor([[1, 2],[3,4]]).share(bob,alice, crypto_provider=crypto_provider)
y = torch.tensor([[2, 0],[0,2]]).share(bob,alice, crypto_provider=crypto_provider)

In [30]:
# matrix multiplication

z = x.mm(y)
z.get()

tensor([[2, 4],
        [6, 8]])

# Verschlüsselter Vergleich
Es ist auch möglich, private Vergleiche zwischen privaten Werten anzustellen. Wir stützen uns hier auf das SecureNN-Protokoll, dessen Details [hier](https://eprint.iacr.org/2018/442.pdf) zu finden sind. Das Ergebnis des Vergleichs ist auch ein privater gemeinsamer Tensor.

In [31]:
x = torch.tensor([25]).share(bob,alice, crypto_provider=crypto_provider)
y = torch.tensor([5]).share(bob,alice, crypto_provider=crypto_provider)

In [32]:
z = x > y
z.get()

tensor([1])

In [33]:
z = x <= y
z.get()

tensor([0])

In [34]:
z = x == y
z.get()

tensor([0])

In [35]:
z = x == y + 20
z.get()

tensor([1])

Sie können auch maximale Operationen ausführen

In [36]:
x = torch.tensor([2, 3, 4, 1]).share(bob,alice, crypto_provider=crypto_provider)
x.max().get()

tensor([4])

In [37]:
x = torch.tensor([[2, 3], [4, 1]]).share(bob,alice, crypto_provider=crypto_provider)
max_values, max_ids = x.max(dim=0)
max_values.get()

tensor([4, 3])

# Herzliche Glückwünsche!!! - Zeit, der Community beizutreten!

Herzlichen Glückwunsch zum Abschluss dieses Notizbuch-Tutorials! Wenn Ihnen dies gefallen hat und Sie sich der Bewegung zur Wahrung der Privatsphäre, zum dezentralen Besitz von KI und der KI-Lieferkette (Daten) anschließen möchten, können Sie dies auf folgende Weise tun!

### Star PySyft auf GitHub

Der einfachste Weg, unserer Community zu helfen, besteht darin, die GitHub-Repos in der Hauptrolle zu spielen! Dies hilft, das Bewusstsein für die coolen Tools zu schärfen, die wir bauen.

- [Star PySyft](https://github.com/OpenMined/PySyft)

### Mach mit bei unserem Slack!

Der beste Weg, um über die neuesten Entwicklungen auf dem Laufenden zu bleiben, ist, sich unserer Community anzuschließen! Sie können dies tun, indem Sie das Formular unter [http://slack.openmined.org](http://slack.openmined.org) ausfüllen.

### Treten Sie einem Code-Projekt bei!

Der beste Weg, um zu unserer Community beizutragen, besteht darin, Code-Mitwirkender zu werden! Sie können jederzeit zur Seite PySyft GitHub Issues gehen und nach "Projekten" filtern. Dies zeigt Ihnen alle Top-Level-Tickets und gibt einen Überblick darüber, an welchen Projekten Sie teilnehmen können! Wenn Sie nicht an einem Projekt teilnehmen möchten, aber ein wenig programmieren möchten, können Sie auch nach weiteren "einmaligen" Miniprojekten suchen, indem Sie nach GitHub-Problemen suchen, die als "gute erste Ausgabe" gekennzeichnet sind.

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### Spenden

Wenn Sie keine Zeit haben, zu unserer Codebasis beizutragen, aber dennoch Unterstützung leisten möchten, können Sie auch Unterstützer unseres Open Collective werden. Alle Spenden fließen in unser Webhosting und andere Community-Ausgaben wie Hackathons und Meetups!

[OpenMined's Open Collective Page](https://opencollective.com/openmined)`m